In [1]:
import pandas as pd
seed_value = 2043
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)

In [2]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}
            
    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping
            

        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)
             
        return X_encoded 

In [4]:
syn_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-100000/synthetic_data_GAN_mWOE_privacy_1_70per_GAN_trained_set0.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head() 

,Unnamed: 0,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,...,infobase,HHstatin,dwllsize,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd
0,-0.222581,-0.466207,-0.222581,0.250208,0.013951,-0.222581,-0.466207,-0.466207,0.013951,-0.222581,...,-0.466207,-0.725060,-0.222581,0.492964,-0.466207,-0.222581,-0.222581,-0.725060,-0.222581,-0.725060
1,-0.222581,0.250208,-0.222581,0.492964,-0.222581,0.492964,-0.222581,-0.222581,0.750182,-0.466207,...,0.013951,0.250208,-0.222581,0.013951,0.013951,-0.222581,0.750182,0.492964,0.750182,0.250208
2,-0.222581,0.013951,-0.466207,0.013951,-0.466207,-0.725060,-0.466207,-0.222581,0.750182,0.013951,...,0.492964,-0.466207,-0.222581,-0.222581,-1.010515,0.013951,0.013951,0.250208,0.492964,-0.222581
3,-0.222581,-0.466207,0.013951,-1.010515,0.013951,-0.222581,-0.222581,0.750182,-0.222581,0.492964,...,-1.010515,-0.466207,0.013951,0.250208,-0.466207,0.492964,-0.466207,0.013951,0.013951,0.250208
4,-0.222581,-0.222581,-0.725060,0.250208,-0.222581,-0.466207,-0.466207,0.013951,-0.222581,0.750182,...,-0.725060,-0.222581,-0.466207,0.013951,-0.466207,-0.725060,-0.222581,0.013951,-0.466207,0.013951


In [5]:
train_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-100000/Original_data_GAN_mWOE_privacy_1_30percent_testing.csv')
y_test=train_df['churn']
X_test=train_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head() 

,Unnamed: 0,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,...,infobase,HHstatin,dwllsize,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd
0,1.437683,0.438523,0.313311,-0.212614,0.023354,0.272117,0.003481,0.177944,0.012381,0.011871,...,-0.033376,-0.045371,-0.056840,0.079190,0.004204,0.001431,0.005313,0.004207,0.005265,-0.028516
1,-0.306999,-0.163961,-0.632909,-0.301970,0.023354,0.003890,0.589992,0.537637,0.012381,0.011871,...,-0.033376,0.074457,0.071067,-0.055289,0.004204,0.001431,0.005313,0.004207,0.005265,0.075098
2,0.003179,0.313311,0.514298,0.332905,0.023354,-0.047286,-0.041586,-0.054922,0.012381,-0.264972,...,-0.033376,0.074457,-0.056840,0.080604,0.004204,0.001431,0.005313,0.004207,0.005265,-0.028516
3,0.313311,-0.166884,-0.139858,-0.212614,0.023354,-0.047286,-0.041586,-0.054922,0.012381,0.011871,...,-0.033376,-0.015970,-0.056840,-0.015729,0.004204,0.001431,0.005313,0.004207,0.005265,0.075098
4,-0.306999,0.003179,0.271556,-0.632909,0.023354,0.049976,-0.727461,-0.798904,0.012381,0.011871,...,-0.033376,0.025427,-0.056840,0.079190,0.004204,0.001431,0.005313,0.004207,0.005265,-0.028516


In [13]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold


#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

12716 2315 1623 13346
pod:  0.8915759235753892
pof:  0.15401503559310759
AUC:  0.8687804439911407
accuracy:  0.8687333333333334


In [14]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

12386 2645 1739 13230
pod:  0.8838265749215044
pof:  0.1759696626970927
AUC:  0.8539284561122058
accuracy:  0.8538666666666667


In [15]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

13702 1329 6714 8255
pod:  0.5514730442915359
pof:  0.0884172709733218
AUC:  0.7315278866591071
accuracy:  0.7319


In [16]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )  
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

11579 3452 1586 13383
pod:  0.8940476985770592
pof:  0.2296587053422926
AUC:  0.8321944966173833
accuracy:  0.8320666666666666


In [17]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier(max_features='sqrt',criterion='mae')
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

C:\Users\IICT\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1284: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). Use criterion='friedman_mse' or 'squared_error' instead, as trees should use a squared error criterion in Gradient Boosting.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is

C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:3

C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:366: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warnings.warn(
C:\Users\IICT\anaconda3\lib\site-packages\sklearn\tree\_classes.py:3

13287 1744 7337 7632
pod:  0.509853697641793
pof:  0.11602687778590912
AUC:  0.6969134099279419
recall:  0.509853697641793
precision:  0.8139931740614335
F1-Score:  0.6269870609981515
accuracy:  0.6973


In [20]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

11933 3098 8674 6295
pod:  0.42053577393279445
pof:  0.2061073780852904
AUC:  0.607214197923752
recall:  0.42053577393279445
precision:  0.6701799212179282
F1-Score:  0.5167884410146951
accuracy:  0.6076


In [21]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier() 

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


14210 821 9652 5317
pod:  0.3552007482129735
pof:  0.05462045106779323
AUC:  0.6502901485725902
recall:  0.3552007482129735
precision:  0.8662430759204953
F1-Score:  0.5038139006016961
accuracy:  0.6509


In [24]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded,y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix
 
tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)    

Epoch 1/30
1750/1750 [==============================] - 9s 5ms/step - loss: 0.0165 - accuracy: 0.9950 - val_loss: 0.0061 - val_accuracy: 0.9981
Epoch 2/30
1750/1750 [==============================] - 8s 5ms/step - loss: 0.0048 - accuracy: 0.9984 - val_loss: 0.0052 - val_accuracy: 0.9982
Epoch 3/30
1750/1750 [==============================] - 8s 5ms/step - loss: 0.0035 - accuracy: 0.9988 - val_loss: 0.0076 - val_accuracy: 0.9974
Epoch 4/30
1750/1750 [==============================] - 8s 5ms/step - loss: 0.0029 - accuracy: 0.9991 - val_loss: 0.0045 - val_accuracy: 0.9982
Epoch 5/30
1750/1750 [==============================] - 8s 5ms/step - loss: 0.0023 - accuracy: 0.9994 - val_loss: 0.0054 - val_accuracy: 0.9984
Epoch 6/30
1750/1750 [==============================] - 8s 5ms/step - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.0071 - val_accuracy: 0.9981
Epoch 7/30
938/938 [==============================] - 2s 2ms/step
12305 2726 1711 13258
pod:  0.8856971073552008
pof:  0.181358525713525